---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [3]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [4]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [5]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''

    with open('university_towns.txt') as f:

        x=f.readlines()

    state_region = []
    curr_state=''

    for line in x:
        line = line.strip()
        if line[-6:] == '[edit]':
                curr_state = line[:-6]
        elif '(' in line:
                region = line[:line.index('(')-1]
                state_region.append([curr_state, region])
        else:
            region = line
            state_region.append([curr_state,region])
    df = pd.DataFrame(state_region, columns=["State", "RegionName"])
    
    return df

In [6]:
get_list_of_university_towns().head()

State    RegionName
0  Alabama        Auburn
1  Alabama      Florence
2  Alabama  Jacksonville
3  Alabama    Livingston
4  Alabama    Montevallo

In [7]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    
    gdp_df = pd.read_excel('gdplev.xls', skiprows = 219)
    gdp_df = gdp_df[['1999q4', 12323.3]]
    gdp_df.columns = ['Quarter', 'GDP']
    for index in range(2, len(gdp_df)):
        if (gdp_df.iloc[index - 2, 1] > gdp_df.iloc[index - 1, 1]) and (gdp_df.iloc[index - 1, 1] > gdp_df.iloc[index, 1]):
            return gdp_df.iloc[index - 1, 0]
    return None

In [8]:
get_recession_start()

'2008q3'

In [9]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    
    gdp_df = pd.read_excel('gdplev.xls', skiprows = 219)
    gdp_df = gdp_df[['1999q4', 12323.3]]
    gdp_df.columns = ['Quarter', 'GDP']
    
    # store the start of recession
    recession_start = get_recession_start()
    # identify the index of that quarter in df
    start = gdp_df[gdp_df['Quarter'] == recession_start].index.tolist()[0]
    # consider the df starting from recession
    gdp_df = gdp_df.iloc[start : ]
    
    # now check for recession end
    for index in range(2, len(gdp_df['Quarter'])):
        if (gdp_df.iloc[index - 2, 1] < gdp_df.iloc[index - 1, 1]) and (gdp_df.iloc[index - 1, 1] < gdp_df.iloc[index, 1]):
            return gdp_df.iloc[index, 0]
    return None


In [10]:
get_recession_end()

'2009q4'

In [11]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    
    # again same, read the exel file
    gdp_df = pd.read_excel('gdplev.xls', skiprows = 219)
    gdp_df = gdp_df[['1999q4', 12323.3]]
    gdp_df.columns = ['Quarter', 'GDP']
    
    # store the start of recession
    recession_start = get_recession_start()
    recession_start_index = gdp_df[gdp_df['Quarter'] == recession_start].index.tolist()[0]
    # store end of recession
    recession_end = get_recession_end()
    recession_end_index = gdp_df[gdp_df['Quarter'] == recession_end].index.tolist()[0]
    
    # consider the recession period only
    gdp_df = gdp_df.iloc[recession_start_index : recession_end_index + 1]
    
    recession_bottom_quarter = None
    recession_bottom_gdp = None
    for index in range(len(gdp_df)):
        if recession_bottom_quarter == None:
            recession_bottom_quarter, recession_bottom_gdp = gdp_df.iloc[index, 0], gdp_df.iloc[index, 1]
        
        elif recession_bottom_gdp > gdp_df.iloc[index, 1]:
            recession_bottom_quarter, recession_bottom_gdp = gdp_df.iloc[index, 0], gdp_df.iloc[index, 1]
        
    return recession_bottom_quarter

In [12]:
get_recession_bottom()

'2009q2'

In [32]:

'''
def convert_housing_data_to_quarters():
    Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    #df = pd.read_csv('City_Zhvi_AllHomes.csv')
    
    # drop unneccessery columns
    #df.drop(['RegionID', 'Metro', 'CountyName', 'SizeRank'], axis = 1, inplace = True)
    #cols = list(df.columns)
    #df.drop(cols[2:47], axis = 1, inplace = True)
    
    #df['State'] = df['State'].map(states)
    
   # yrs = list(range(2000, 2017))
    #qtr = ['q1', 'q2', 'q3', 'q4']
    #new_cols = [str(yr)+ q for yr in yrs for q in qtr]
    
    #cols = list(df.columns)
    #convert_qarter = 2
    #for i in new_cols:
     #   df[i] = np.mean()
    #return df
    
def new_col_names():
    #generating the new coloumns names 
    years = list(range(2000,2017))
    quars = ['q1','q2','q3','q4']
    quar_years = []
    for i in years:
        for x in quars:
            quar_years.append((str(i)+x))
    return quar_years[:67]
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    data = pd.read_csv('City_Zhvi_AllHomes.csv')
    data.drop(['Metro','CountyName','RegionID','SizeRank'],axis=1,inplace=True)
    data['State'] = data['State'].map(states)
    data.set_index(['State','RegionName'],inplace=True)
    col = list(data.columns)
    col = col[0:45]
    data.drop(col,axis=1,inplace=True)

    #qs is the quarters of the year
    qs = [list(data.columns)[x:x+3] for x in range(0, len(list(data.columns)), 3)]
    
    # new columns
    column_names = new_col_names()
    for col,q in zip(column_names,qs):
        data[col] = data[q].mean(axis=1)
        
    data = data[column_names]
    return data

In [33]:

convert_housing_data_to_quarters()


2000q1        2000q2  \
State                RegionName                                     
New York             New York          2.491333e+05  2.564333e+05   
California           Los Angeles       2.070667e+05  2.144667e+05   
Illinois             Chicago           1.384000e+05  1.436333e+05   
Pennsylvania         Philadelphia      5.300000e+04  5.363333e+04   
Arizona              Phoenix           1.118333e+05  1.143667e+05   
Nevada               Las Vegas         1.326000e+05  1.343667e+05   
California           San Diego         2.229000e+05  2.343667e+05   
Texas                Dallas            8.446667e+04  8.386667e+04   
California           San Jose          3.743000e+05  4.065667e+05   
Florida              Jacksonville      8.860000e+04  8.970000e+04   
California           San Francisco     4.305000e+05  4.644667e+05   
Texas                Austin            1.429667e+05  1.452667e+05   
Michigan             Detroit           6.620000e+04  6.830000e+04   
Ohio                 Columbus          9.436667e+04  9.583333e+04   
Tennessee            Memphis           7.250000e+04  7.320000e+04   
North Carolina       Charlotte         1.269333e+05  1.283667e+05   
Texas                El Paso           7.626667e+04  7.686667e+04   
Massachusetts        Boston            2.069333e+05  2.191667e+05   
Washington           Seattle           2.486000e+05  2.556000e+05   
Maryland             Baltimore         5.966667e+04  5.950000e+04   
Colorado             Denver            1.622333e+05  1.678333e+05   
District of Columbia Washington        1.377667e+05  1.442000e+05   
Tennessee            Nashville         1.138333e+05  1.152667e+05   
Wisconsin            Milwaukee         7.803333e+04  7.906667e+04   
Arizona              Tucson            1.018333e+05  1.029667e+05   
Oregon               Portland          1.528000e+05  1.547667e+05   
Oklahoma             Oklahoma City     7.643333e+04  7.750000e+04   
Nebraska             Omaha             1.128000e+05  1.141000e+05   
New Mexico           Albuquerque       1.258667e+05  1.267000e+05   
California           Fresno            9.410000e+04  9.526667e+04   
...                                             ...           ...   
New Hampshire        East Hampstead    1.618000e+05  1.691000e+05   
Minnesota            Emily                      NaN           NaN   
Missouri             Garden City       1.224667e+05  1.225000e+05   
Wisconsin            Hager City        1.339333e+05  1.332000e+05   
Arkansas             Mountainburg      5.716667e+04  6.433333e+04   
Wisconsin            Oostburg          1.072667e+05  1.081000e+05   
New York             Upper Brookville  1.230967e+06  1.230967e+06   
Nebraska             Utica             8.890000e+04  9.193333e+04   
Hawaii               Volcano           9.870000e+04  1.053667e+05   
South Carolina       Wedgefield                 NaN           NaN   
Michigan             Williamston       1.591667e+05  1.613000e+05   
Arkansas             Decatur           6.360000e+04  6.440000e+04   
Indiana              Edgewood          9.170000e+04  9.186667e+04   
Tennessee            Palmyra                    NaN           NaN   
Maryland             Saint Inigoes     1.480667e+05  1.476000e+05   
Indiana              Marysville                 NaN           NaN   
California           Forest Falls      1.135333e+05  1.144000e+05   
Michigan             Greilickville              NaN           NaN   
Minnesota            McGrath                    NaN           NaN   
Missouri             Bois D Arc        1.078000e+05  1.070000e+05   
Virginia             Henrico           1.285667e+05  1.307667e+05   
New Jersey           Diamond Beach     1.739667e+05  1.831000e+05   
Wisconsin            Lake Ripley       1.457667e+05  1.448333e+05   
Pennsylvania         Highland          1.325000e+05  1.321333e+05   
                     Palmyra           9.046667e+04  9.026667e+04   
Wisconsin             Wrightstown      1.01

In [28]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    data = convert_housing_data_to_quarters().copy()
    data = data.loc[:,'2008q3':'2009q2']
    data = data.reset_index()
    def price_ratio(row):
        return (row['2008q3'] - row['2009q2'])/row['2008q3']
    
    data['up&down'] = data.apply(price_ratio,axis=1)
    #uni data 
    
    uni_town = get_list_of_university_towns()['RegionName']
    uni_town = set(uni_town)

    def is_uni_town(row):
        #check if the town is a university towns or not.
        if row['RegionName'] in uni_town:
            return 1
        else:
            return 0
    data['is_uni'] = data.apply(is_uni_town,axis=1)
    
    
    not_uni = data[data['is_uni']==0].loc[:,'up&down'].dropna()
    is_uni  = data[data['is_uni']==1].loc[:,'up&down'].dropna()
    def better():
        if not_uni.mean() < is_uni.mean():
            return 'non-university town'
        else:
            return 'university town'
    p_val = list(ttest_ind(not_uni, is_uni))[1]
    result = (True,p_val,better())
    return result

In [29]:
run_ttest()

(True, 4.481521388099153e-05, 'university town')